**Siamese Network One-Shot**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import  numpy as np

In [9]:
df = pd.read_csv('Siamese_Training_1.csv')
labels = df['Label'].values
raw_specs = df.drop(columns=['Label']).values.astype(float)

In [10]:
lam, p, niter = 1e4, 0.01, 10
def baseline_als(y):
    L = len(y)
    D = np.diff(np.eye(L), 2)
    D = lam * D.dot(D.T)
    w = np.ones(L)
    for _ in range(niter):
        b = np.linalg.solve(np.diag(w) + D, w * y)
        w = p * (y > b) + (1 - p) * (y < b)
    return b

def preprocess(arr):
    out = np.zeros_like(arr)
    for i, s in enumerate(arr):
        b = baseline_als(s)
        c = s - b
        norm = np.linalg.norm(c)
        out[i] = c / norm if norm > 0 else c
    return out

In [11]:
spectra = preprocess(raw_specs)

In [12]:
def augment(spec, noise_std=0.01, shift_max=2):
    spec_noisy = spec + np.random.normal(0, noise_std, size=spec.shape)
    shift = np.random.randint(-shift_max, shift_max + 1)
    return np.roll(spec_noisy, shift)

# 3. Pair dataset for contrastive learning
class RamanPairDataset(Dataset):
    def __init__(self, specs, labels, augment_fn=None):
        self.specs = specs
        self.labels = labels
        self.augment = augment_fn
        self.by_label = {c: np.where(labels == c)[0] for c in np.unique(labels)}

    def __len__(self):
        return len(self.specs)

    def __getitem__(self, idx):
        x1 = self.specs[idx]
        y1 = self.labels[idx]
        if np.random.rand() < 0.5:
            j = np.random.choice(self.by_label[y1])
            label = 1.0
        else:
            neg = [c for c in self.by_label if c != y1]
            y2 = np.random.choice(neg)
            j = np.random.choice(self.by_label[y2])
            label = 0.0
        x2 = self.specs[j]
        if self.augment:
            x1 = self.augment(x1)
            x2 = self.augment(x2)
        return (torch.tensor(x1, dtype=torch.float32).unsqueeze(0),
                torch.tensor(x2, dtype=torch.float32).unsqueeze(0),
                torch.tensor(label, dtype=torch.float32))

# 4. Siamese network definition
class SiameseNet(nn.Module):
    def __init__(self, input_len, embed_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=7, padding=3), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=5, padding=2), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((input_len // 4) * 32, embed_dim),
            nn.ReLU()
        )

    def forward(self, x):
        z = self.encoder(x)
        return F.normalize(z, dim=1)

# 5. Contrastive loss
def contrastive_loss(z1, z2, label, margin=1.0):
    dist = F.pairwise_distance(z1, z2)
    loss_pos = label * dist**2
    loss_neg = (1 - label) * F.relu(margin - dist)**2
    return (loss_pos + loss_neg).mean()

In [17]:
input_len = spectra.shape[1]
dataset = RamanPairDataset(spectra, labels, augment_fn=augment)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = SiameseNet(input_len, embed_dim=64)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()
for epoch in range(100):
    total_loss = 0.0
    for x1, x2, lbl in loader:
        z1, z2 = model(x1), model(x2)
        loss = contrastive_loss(z1, z2, lbl)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x1.size(0)
    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "siamese_raman.pth")

Epoch 1, Loss: 0.3728
Epoch 2, Loss: 0.4802
Epoch 3, Loss: 0.4129
Epoch 4, Loss: 0.4454
Epoch 5, Loss: 0.6235
Epoch 6, Loss: 0.5724
Epoch 7, Loss: 0.4399
Epoch 8, Loss: 0.5307
Epoch 9, Loss: 0.4477
Epoch 10, Loss: 0.4405
Epoch 11, Loss: 0.4490
Epoch 12, Loss: 0.3583
Epoch 13, Loss: 0.2981
Epoch 14, Loss: 0.4502
Epoch 15, Loss: 0.3504
Epoch 16, Loss: 0.4998
Epoch 17, Loss: 0.4730
Epoch 18, Loss: 0.4418
Epoch 19, Loss: 0.4323
Epoch 20, Loss: 0.5982
Epoch 21, Loss: 0.4678
Epoch 22, Loss: 0.4132
Epoch 23, Loss: 0.3513
Epoch 24, Loss: 0.4048
Epoch 25, Loss: 0.3779
Epoch 26, Loss: 0.4366
Epoch 27, Loss: 0.4285
Epoch 28, Loss: 0.3266
Epoch 29, Loss: 0.2495
Epoch 30, Loss: 0.2891
Epoch 31, Loss: 0.3133
Epoch 32, Loss: 0.2789
Epoch 33, Loss: 0.4512
Epoch 34, Loss: 0.3092
Epoch 35, Loss: 0.3931
Epoch 36, Loss: 0.4782
Epoch 37, Loss: 0.4790
Epoch 38, Loss: 0.4485
Epoch 39, Loss: 0.4152
Epoch 40, Loss: 0.4229
Epoch 41, Loss: 0.3815
Epoch 42, Loss: 0.3039
Epoch 43, Loss: 0.3302
Epoch 44, Loss: 0.26